In [ ]:
import numpy as np 
import torch, matplotlib
import matplotlib.pyplot as plt 
from scipy.ndimage import maximum_filter, label, generate_binary_structure

from utils import *

## 0. Experiment Settings

- **Number of sources:** 7 
- **Number of sensors:** 36
- **Number of snapshots:** 1000  
- **SNR:** 20 dB  
- **Azimuth range:** $0$ to $\pi$  
- **Elevation range:** $0$ to $\pi/2$  


In [ ]:
d = 7
t = 200
snr = 10

phi = torch.rand(d) * torch.pi # random azimuth angles
theta = torch.rand(d) * torch.pi / 2 # random elevation angles

source_std = torch.eye(d, dtype=torch.complex64)
noise_std = torch.sqrt(torch.sum(torch.diag(source_std) ** 2)) * 10 ** (-snr/20)

S = source_std @ torch.randn(d, t, dtype=torch.complex64) / sqrt(2) # sources signal

## 1. Uniform Rectangular Array

#### 1.1. Define Uniform Rectangular Array

In [ ]:
lamda = 0.2
min_distance = 0.1

size_horizontal = 6
size_vertical = 6

ura = URA(lamda, min_distance, size_horizontal, size_vertical)
ura.build_array_manifold()
ura.plot()

#### 1.2. Apply MUSIC for Uniform Rectangular Array

In [ ]:
N1 = noise_std * torch.randn(ura.nbSensors, t) / sqrt(2)
X1 = ura.get_steering_vector(phi, theta) @ S + N1

cov = X1 @ X1.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_ura = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), ura.array_manifold), dim=0) ** 2

## 2. 2D Nested Array

#### 2.1. Define 2D Nested Array

In [ ]:
levels_horizontal = [3, 3]
levels_vertical = [3, 3]

na = NestedArray2D(lamda, min_distance, levels_horizontal, levels_vertical)
na.build_array_manifold()
na.plot()

#### 2.2. Apply MUSIC for 2D Nested Array

In [ ]:
N2 = noise_std * torch.randn(na.nbSensors, t) / sqrt(2)
X2 = na.get_steering_vector(phi, theta) @ S + N2

cov = X2 @ X2.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_na = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), na.array_manifold), dim=0) ** 2

## 3. Open Box Array

#### 3.1 Define Open Box Array

In [ ]:
size_horizontal_oba = 16
size_vertical_oba = 11

oba = OpenBoxArray(lamda, min_distance, size_horizontal_oba, size_vertical_oba)
oba.build_array_manifold()
oba.plot()

#### 3.2 Apply MUSIC for Open Box Array

In [ ]:
N3 = noise_std * torch.randn(oba.nbSensors, t) / sqrt(2)
X3 = oba.get_steering_vector(phi, theta) @ S + N3

cov = X3 @ X3.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_oba = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), oba.array_manifold), dim=0) ** 2

## 4. Half Open Box Array

#### 4.1. Define Half Open Box Array

In [ ]:
size_horizontal_hoba = 16
size_vertical_hoba = 11

hoba = HalfOpenBoxArray(lamda, min_distance, size_horizontal_hoba, size_vertical_hoba)
hoba.build_array_manifold()
hoba.plot()

#### 4.2 Apply MUSIC for Half Open Box Array

In [ ]:
N4 = noise_std * torch.randn(oba.nbSensors, t) / sqrt(2)
X4 = hoba.get_steering_vector(phi, theta) @ S + N4

cov = X4 @ X4.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_hoba = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), hoba.array_manifold), dim=0) ** 2

## 5. Hexagonal Lattice Array 

#### 5.1. Define Hexagonal Lattice Array 

In [ ]:
size_radius = 4

hla = HexagonalLatticeArray(lamda, min_distance*sqrt(2), size_radius)
hla.build_array_manifold()
hla.plot()

#### 5.2. Apply MUSIC for Hexagonal Lattice Array

In [ ]:
N5 = noise_std * torch.randn(hla.nbSensors, t) / sqrt(2)
X5 = hla.get_steering_vector(phi, theta) @ S + N5

cov = X5 @ X5.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_hla = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), hla.array_manifold), dim=0) ** 2

In [ ]:
size_radius = 6

ohla = OpenHexagonalLatticeArray(lamda, min_distance*sqrt(2), size_radius)
ohla.build_array_manifold()
ohla.plot()

In [ ]:
N6 = noise_std * torch.randn(ohla.nbSensors, t) / sqrt(2)
X6 = ohla.get_steering_vector(phi, theta) @ S + N6

cov = X6 @ X6.T.conj() / t
vals, vecs = torch.linalg.eigh(cov)
En = vecs[:, :-d]

spectrum_ohla = 1 / torch.norm(torch.einsum('mi,mjk->ijk', En.conj(), ohla.array_manifold), dim=0) ** 2

## 6. Plot Results

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(15, 12))

plot_MUSIC_spectrum(fig, axes[0, 0], "Uniform Rectangular Array", spectrum_ura, ura.phi_space, ura.theta_space, phi, theta)
plot_MUSIC_spectrum(fig, axes[0, 1], "2D Nested Array", spectrum_na, na.phi_space, na.theta_space, phi, theta)
plot_MUSIC_spectrum(fig, axes[1, 0], "Open Box Array", spectrum_oba, oba.phi_space, oba.theta_space, phi, theta)
plot_MUSIC_spectrum(fig, axes[1, 1], "Half Open Box Array", spectrum_hoba, hoba.phi_space, hoba.theta_space, phi, theta)
plot_MUSIC_spectrum(fig, axes[2, 0], "Hexagonal Lattice Array", spectrum_hla, hla.phi_space, hla.theta_space, phi, theta)
plot_MUSIC_spectrum(fig, axes[2, 1], "Open Hexagonal Lattice Array", spectrum_ohla, ohla.phi_space, ohla.theta_space, phi, theta)

plt.tight_layout()
plt.show()

## 7. Monte Carlo simulation

In [ ]:
d = 7
t = 200
mc = 1000
SNRs = [0, 5, 10, 15, 20]

methods = ["URA", "2D Nested Array", "Open Box Array", "Half Open Box Array", "Hexagonal Lattice Array"]
values = {
    "URA": [],
    "2D Nested Array": [],
    "Open Box Array": [],
    "Half Open Box Array": [],
    "Hexagonal Lattice Array": []
}

rmspe = RMSPE(d)

for snr in SNRs:

    rmspe_ura, rmspe_na, rmspe_oba, rmspe_hoba, rmspe_hla = [], [], [], [], []

    for _ in range(mc):

        phi_true = generate_angles(d, 0, torch.pi)
        theta_true = generate_angles(d, 0, torch.pi/2)

        source_std = torch.eye(d, dtype=torch.complex64)
        noise_std = torch.sqrt(torch.sum(torch.diag(source_std) ** 2)) * 10 ** (-snr/20)

        S = source_std @ torch.randn(d, t, dtype=torch.complex64) / sqrt(2) 

        N1 = noise_std * torch.randn(ura.nbSensors, t) / sqrt(2)
        X1 = ura.get_steering_vector(phi_true, theta_true) @ S + N1
        phi_ura, theta_ura = MUSIC(X1, d, ura)
        rmspe_ura.append(rmspe.calculate(phi_ura, theta_ura, phi_true, theta_true))

        N2 = noise_std * torch.randn(na.nbSensors, t) / sqrt(2)
        X2 = na.get_steering_vector(phi_true, theta_true) @ S + N2
        phi_na, theta_na = MUSIC(X2, d, na)
        rmspe_na.append(rmspe.calculate(phi_na, theta_na, phi_true, theta_true))

        N3 = noise_std * torch.randn(oba.nbSensors, t) / sqrt(2)
        X3 = oba.get_steering_vector(phi_true, theta_true) @ S + N3
        phi_oba, theta_oba = MUSIC(X3, d, oba)
        rmspe_oba.append(rmspe.calculate(phi_oba, theta_oba, phi_true, theta_true))

        N4 = noise_std * torch.randn(hoba.nbSensors, t) / sqrt(2)
        X4 = hoba.get_steering_vector(phi_true, theta_true) @ S + N4
        phi_hoba, theta_hoba = MUSIC(X4, d, hoba)
        rmspe_hoba.append(rmspe.calculate(phi_hoba, theta_hoba, phi_true, theta_true))

        N5 = noise_std * torch.randn(hla.nbSensors, t) / sqrt(2)
        X5 = hla.get_steering_vector(phi_true, theta_true) @ S + N5
        phi_hla, theta_hla = MUSIC(X5, d, hla)
        rmspe_hla.append(rmspe.calculate(phi_hla, theta_hla, phi_true, theta_true))

    values['URA'].append(sum(rmspe_ura) / mc)
    values['2D Nested Array'].append(sum(rmspe_na) / mc)
    values['Open Box Array'].append(sum(rmspe_oba) / mc)
    values['Half Open Box Array'].append(sum(rmspe_hoba) / mc)
    values['Hexagonal Lattice Array'].append(sum(rmspe_hla) / mc)

for method in methods:
    plt.plot(SNRs, values[method], marker='o', label=method)

plt.title("Performance Comparison Across SNR Levels", fontsize=14)
plt.xlabel("SNR (dB)", fontsize=12)
plt.ylabel("RMSPE (rad)", fontsize=12)
plt.xticks(SNRs)
plt.grid(True, which="both", linestyle="--", linewidth=0.5)
plt.legend(title="Methods")
plt.tight_layout()

plt.show()